In [1]:
!pip install ipywidgets

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [32]:
import pandas as pd
from pandasgui import show
import ipywidgets as widgets
from IPython.display import display
import streamlit as st



In [63]:
# Load your NER dataset
path = r''
df = pd.read_csv(path)
df['tokens'] = df['tokens'].apply(lambda x: x.split('||'))
df['annotations'] = df['annotations'].apply(lambda x: x.split('||'))
df

,sentence,tokens,annotations
0,Factory Buys 32cm Euro Top Mattress - King Ski...,"[Factory, Buys, 32cm, Euro, Top, Mattress, -, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Home Furniture Home Furniture Bedroom Dining L...,"[Home, Furniture, Home, Furniture, Bedroom, Di...","[B-product, I-product, O, O, B-product, I-prod..."
2,LEARN MORE 4.9 Rated 4.9 out of 5 stars Based ...,"[LEARN, MORE, 4, ., 9, Rated, 4, ., 9, out, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Unit price / per Add Sale Ships for Free 4 % O...,"[Unit, price, /, per, Add, Sale, Ships, for, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,Unit price / per Add Sale Australia Owned & St...,"[Unit, price, /, per, Add, Sale, Australia, Ow...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
1170,Products | Guaranteed A Fine Furniture | High-...,"[Products, , |Guaranteed, A, Fine, Furniture, ...","[O, O, O, O, O, B-product, O, O, O, O, O, O, O..."
1171,Our furniture selection Guaranteed A Furniture...,"[Our, furniture, selection, Guaranteed, A, Fur...","[O, B-product, O, O, O, O, O, O, O, O, O, O, O..."
1172,BEDROOM BEDROOM BEDROOM BEDROOM BEDROOM BEDROO...,"[BEDROOM, BEDROOM, BEDROOM, BEDROOM, BEDROOM, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1173,Regular Hours Monday - Friday | 10:00am - 7:00...,"[Regular, Hours, Monday, -, Friday, , |10, :, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [64]:
print(df['annotations'].explode().value_counts())

annotations
O            150334
B-product      2563
I-product      1137
Name: count, dtype: int64


In [62]:
df['tokens'] = df['tokens'].apply(lambda x: x.split('||'))
df['annotations'] = df['annotations'].apply(lambda x: x.split('||'))

AttributeError: 'list' object has no attribute 'split'

In [65]:
# Initialize the current row index and token chunk index
current_row = widgets.IntText(value=0, description="Row:", disabled=True)
chunk_size = 20  # Set the number of tokens to display at a time
current_chunk = widgets.IntText(value=0, description="Chunk:", disabled=True)

# Create a container for the editing widgets
edit_container = widgets.VBox()

# Function to edit tokens and annotations in chunks
def edit_ner_row(row_idx, chunk_idx=0):
    # Clear the previous widgets in the container
    edit_container.children = []
    
    row = df.iloc[row_idx]
    
    # Get the chunk of tokens and annotations to display
    start = chunk_idx * chunk_size
    end = min(start + chunk_size, len(row['tokens']))
    tokens_chunk = row['tokens'][start:end]
    annotations_chunk = row['annotations'][start:end]
    
    token_widgets = []
    annotation_widgets = []
    
    for token, annotation in zip(tokens_chunk, annotations_chunk):
        token_field = widgets.Text(
            value=token,
            description='Token:',
            disabled=False,
            layout=widgets.Layout(width='40%')
        )
        
        annotation_field = widgets.Text(
            value=annotation,
            description='Annotation:',
            disabled=False,
            layout=widgets.Layout(width='40%')
        )
        
        # Add token and annotation to the list
        token_widgets.append(token_field)
        annotation_widgets.append(annotation_field)
    
    save_button = widgets.Button(description="Save Changes")
    
    def save_changes(_):
        # Update only the chunk of tokens and annotations back to the DataFrame
        new_tokens = [token_field.value for token_field in token_widgets]
        new_annotations = [annotation_field.value for annotation_field in annotation_widgets]
        
        df.at[row_idx, 'tokens'][start:end] = new_tokens
        df.at[row_idx, 'annotations'][start:end] = new_annotations
        
        print(f"Row {row_idx + 1}, Chunk {chunk_idx + 1} updated.")
    
    save_button.on_click(save_changes)
    
    # Add the new widgets (tokens and annotations in pairs) to the container
    rows = [widgets.HBox([token_field, annotation_field]) for token_field, annotation_field in zip(token_widgets, annotation_widgets)]
    
    # Add navigation buttons for chunking (Previous/Next Chunk)
    chunk_nav_buttons = []
    if chunk_idx > 0:
        prev_chunk_button = widgets.Button(description="Previous Chunk")
        prev_chunk_button.on_click(lambda _: edit_ner_row(row_idx, chunk_idx - 1))
        chunk_nav_buttons.append(prev_chunk_button)
    
    if end < len(row['tokens']):
        next_chunk_button = widgets.Button(description="Next Chunk")
        next_chunk_button.on_click(lambda _: edit_ner_row(row_idx, chunk_idx + 1))
        chunk_nav_buttons.append(next_chunk_button)
    
    # Add the save button and rows to the container
    edit_container.children = rows + [save_button, widgets.HBox(chunk_nav_buttons)]

# Function to move to the next row
def next_row(change):
    current_row.value += 1
    if current_row.value >= len(df):
        current_row.value = len(df) - 1  # Keep it within bounds
    current_chunk.value = 0  # Reset to the first chunk of the new row
    edit_ner_row(current_row.value, current_chunk.value)

# Button to go to the next row
next_row_button = widgets.Button(description="Next Row")
next_row_button.on_click(next_row)

# Initial display for the first row and chunk
edit_ner_row(current_row.value, current_chunk.value)

# Display the "Next Row" button and the container with widgets
display(widgets.VBox([next_row_button, current_row, edit_container]))

Row 1, Chunk 1 updated.
Row 2, Chunk 1 updated.
Row 2, Chunk 2 updated.
Row 2, Chunk 1 updated.
Row 2, Chunk 3 updated.
Row 2, Chunk 4 updated.
Row 2, Chunk 4 updated.
Row 2, Chunk 5 updated.
Row 2, Chunk 6 updated.
Row 2, Chunk 7 updated.
Row 2, Chunk 7 updated.
Row 2, Chunk 8 updated.
Row 2, Chunk 9 updated.
Row 2, Chunk 10 updated.
Row 2, Chunk 11 updated.
Row 2, Chunk 12 updated.
Row 2, Chunk 13 updated.
Row 2, Chunk 13 updated.
Row 2, Chunk 14 updated.
Row 2, Chunk 15 updated.
Row 2, Chunk 16 updated.
Row 2, Chunk 17 updated.
Row 2, Chunk 18 updated.
Row 2, Chunk 19 updated.
Row 2, Chunk 19 updated.
Row 2, Chunk 20 updated.
Row 2, Chunk 20 updated.
Row 2, Chunk 21 updated.
Row 2, Chunk 22 updated.
Row 2, Chunk 23 updated.
Row 2, Chunk 24 updated.
Row 2, Chunk 25 updated.
Row 2, Chunk 25 updated.
Row 2, Chunk 25 updated.
Row 2, Chunk 26 updated.
Row 2, Chunk 26 updated.
Row 2, Chunk 26 updated.
Row 2, Chunk 27 updated.
Row 2, Chunk 28 updated.
Row 2, Chunk 28 updated.
Row 2, Chunk 

In [75]:
print(df['annotations'].explode().value_counts())
# I-product v
# B-product

annotations
O            149724
B-product      2915
I-product      1395
Name: count, dtype: int64


In [76]:
df

,sentence,tokens,annotations
0,Factory Buys 32cm Euro Top Mattress - King Ski...,"[Factory, Buys, 32cm, Euro, Top, Mattress, -, ...","[O, O, O, O, O, B-product, O, O, O, O, O, O, O..."
1,Home Furniture Home Furniture Bedroom Dining L...,"[Home, Furniture, Home, Furniture, Bedroom, Di...","[B-product, I-product, B-product, I-product, B..."
2,LEARN MORE 4.9 Rated 4.9 out of 5 stars Based ...,"[LEARN, MORE, 4, ., 9, Rated, 4, ., 9, out, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Unit price / per Add Sale Ships for Free 4 % O...,"[Unit, price, /, per, Add, Sale, Ships, for, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,Unit price / per Add Sale Australia Owned & St...,"[Unit, price, /, per, Add, Sale, Australia, Ow...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
1170,Products | Guaranteed A Fine Furniture | High-...,"[Products, , |Guaranteed, A, Fine, Furniture, ...","[O, O, O, O, O, B-product, O, O, O, O, O, O, O..."
1171,Our furniture selection Guaranteed A Furniture...,"[Our, furniture, selection, Guaranteed, A, Fur...","[O, B-product, O, O, O, O, O, O, O, O, O, O, O..."
1172,BEDROOM BEDROOM BEDROOM BEDROOM BEDROOM BEDROO...,"[BEDROOM, BEDROOM, BEDROOM, BEDROOM, BEDROOM, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1173,Regular Hours Monday - Friday | 10:00am - 7:00...,"[Regular, Hours, Monday, -, Friday, , |10, :, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [77]:
df['tokens'] = df['tokens'].apply(lambda x: '||'.join(x))
df['annotations'] = df['annotations'].apply(lambda x: '||'.join(x))

df.to_csv('updated_ner_dataset.csv', index=False)


In [56]:
# Load your NER dataset
path = r''
df_updated = pd.read_csv(path)
df_updated['tokens'] = df_updated['tokens'].apply(lambda x: x.split('||'))
df_updated['annotations'] = df_updated['annotations'].apply(lambda x: x.split('||'))
df_updated

,sentence,tokens,annotations
0,Factory Buys 32cm Euro Top Mattress - King Ski...,"[Factory, Buys, 32cm, Euro, Top, Mattress, -, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Home Furniture Home Furniture Bedroom Dining L...,"[Home, Furniture, Home, Furniture, Bedroom, Di...","[B-product, I-product, O, O, B-product, I-prod..."
2,LEARN MORE 4.9 Rated 4.9 out of 5 stars Based ...,"[LEARN, MORE, 4, ., 9, Rated, 4, ., 9, out, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Unit price / per Add Sale Ships for Free 4 % O...,"[Unit, price, /, per, Add, Sale, Ships, for, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,Unit price / per Add Sale Australia Owned & St...,"[Unit, price, /, per, Add, Sale, Australia, Ow...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
1170,Products | Guaranteed A Fine Furniture | High-...,"[Products, , |Guaranteed, A, Fine, Furniture, ...","[O, O, O, O, O, B-product, O, O, O, O, O, O, O..."
1171,Our furniture selection Guaranteed A Furniture...,"[Our, furniture, selection, Guaranteed, A, Fur...","[O, B-product, O, O, O, O, O, O, O, O, O, O, O..."
1172,BEDROOM BEDROOM BEDROOM BEDROOM BEDROOM BEDROO...,"[BEDROOM, BEDROOM, BEDROOM, BEDROOM, BEDROOM, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1173,Regular Hours Monday - Friday | 10:00am - 7:00...,"[Regular, Hours, Monday, -, Friday, , |10, :, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [57]:
print(df_updated['annotations'].explode().value_counts())


annotations
O            150334
B-product      2563
I-product      1137
Name: count, dtype: int64
